In [1]:
%pip install pandas
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')

Note: you may need to restart the kernel to use updated packages.


C:\Users\kar\AppData\Local\Temp\ipykernel_9788\2285144985.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
from compile import get_train_data, get_test_data

In [4]:
train_image_arr = get_train_data()[0]
test_image_arr = get_test_data()[0]
train_label = get_train_data()[-1]
test_label = get_test_data()[-1]

In [5]:
from keras.utils import to_categorical

transdict = {1: 0, 2: 1, 3: 2, 4: 3}
train_label = [transdict.get(i, i) for i in train_label]
test_label= [transdict.get(i, i) for i in test_label]
train_label = to_categorical(train_label)
test_label = to_categorical(test_label)
train_image_arr = np.array(train_image_arr)
test_image_arr = np.array(test_image_arr)

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

model = Sequential()
model.add(Dense(units=128, activation='relu', input_shape=train_image_arr[0].shape))
model.add(Dense(units=64, activation='softmax'))
model.add(Flatten())
model.add(Dense(units=4, activation='softmax'))
model.compile(optimizer="adam",loss="categorical_crossentropy", metrics=['accuracy'])

In [8]:
import keras
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="optimal_model_weights.h5",
        save_best_only = "True",
        monitor="val_loss"),
    keras.callbacks.EarlyStopping(patience=9, restore_best_weights=True)
]
history = model.fit(train_image_arr, train_label, epochs = 5, validation_data = (test_image_arr, test_label), callbacks = callbacks)

Epoch 1/5


 1/62 [..............................] - ETA: 9:10 - loss: 1.3954 - accuracy: 0.1562

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'])
plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()